# 🛡️ GHAS Code Scanning Demo — CodeQL CLI & GitHub CLI
**Globomantics Robot Fleet Manager** | Modules 5 & 6

This notebook demonstrates CodeQL CLI and GitHub CLI commands for managing code scanning alerts, inspecting results, and triaging vulnerabilities.

In [1]:
import subprocess
import json
import os

REPO = "timothywarner-org/globomantics-robot-fleet"
REPO_DIR = r"C:\github\globomantics-robot-fleet"

# Ensure CodeQL CLI is in PATH for subprocess calls
os.environ["PATH"] = r"C:\codeql-home\codeql" + os.pathsep + os.environ["PATH"]

def run(cmd, shell="pwsh", cwd=REPO_DIR):
    """Run a command and display output with formatting."""
    print(f"\u26a1 Running: {cmd}\n{'\u2500' * 60}")
    result = subprocess.run(
        [shell, "-NoProfile", "-Command", cmd],
        capture_output=True, text=True, cwd=cwd,
        env={**os.environ, "NO_COLOR": "1"}
    )
    if result.stdout.strip():
        print(result.stdout.strip())
    if result.stderr.strip():
        print(f"\u26a0\ufe0f {result.stderr.strip()}")
    print(f"{'\u2500' * 60}")
    return result

def run_gh(cmd, cwd=REPO_DIR):
    """Run a gh/git command directly."""
    print(f"\u26a1 Running: {cmd}\n{'\u2500' * 60}")
    result = subprocess.run(
        cmd, shell=True, capture_output=True, text=True, cwd=cwd,
        env={**os.environ, "NO_COLOR": "1"}
    )
    if result.stdout.strip():
        print(result.stdout.strip())
    if result.stderr.strip():
        print(f"\u26a0\ufe0f {result.stderr.strip()}")
    print(f"{'\u2500' * 60}")
    return result

## 🔧 Pre-flight Checks

Verify that all required CLI tools are installed and authenticated before running demo commands.

In [2]:
# Verify CodeQL CLI is installed
run_gh("codeql --version")

# Verify GitHub CLI is authenticated
run_gh("gh auth status")

# Verify we are in a valid git repo and show current commit
run_gh(f"git -C {REPO_DIR} rev-parse HEAD")

⚡ Running: codeql --version
────────────────────────────────────────────────────────────


CodeQL command-line toolchain release 2.24.2.
Copyright (C) 2019-2026 GitHub, Inc.
Unpacked in: C:\codeql-home\codeql
   Analysis results depend critically on separately distributed query and
   extractor modules. To list modules that are visible to the toolchain,
   use 'codeql resolve packs' and 'codeql resolve languages'.
────────────────────────────────────────────────────────────
⚡ Running: gh auth status
────────────────────────────────────────────────────────────


github.com
  âœ“ Logged in to github.com account timothywarner (keyring)
  - Active account: true
  - Git operations protocol: https
  - Token: gho_************************************
  - Token scopes: 'admin:org', 'admin:public_key', 'delete_repo', 'gist', 'repo', 'workflow'
────────────────────────────────────────────────────────────
⚡ Running: git -C C:\github\globomantics-robot-fleet rev-parse HEAD
────────────────────────────────────────────────────────────
6166c707fbc2cfa80b2005ee550bb91e2bb1245b
────────────────────────────────────────────────────────────


CompletedProcess(args='git -C C:\\github\\globomantics-robot-fleet rev-parse HEAD', returncode=0, stdout='6166c707fbc2cfa80b2005ee550bb91e2bb1245b\n', stderr='')

## 🔍 CodeQL Default Setup Configuration

CodeQL **default setup** is GitHub's zero-config option. You don't write a workflow file — GitHub detects the languages, picks the query suite, and runs analysis automatically.

> **💡 GH-500 Exam Tip:** "Default setup" and "default suite" are **different things**. Default setup is the zero-config enablement mode. The default suite is a specific set of queries (fewer, lower noise). Default setup lets you choose *either* the default suite or the security-extended suite.

In [3]:
# Check CodeQL analysis configuration (default setup)
run_gh(f'gh api repos/{REPO}/code-scanning/default-setup --jq "."')

# List recent CodeQL workflow runs
run_gh(f'gh run list --repo {REPO} --workflow=codeql.yml --limit=5')

⚡ Running: gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/default-setup --jq "."
────────────────────────────────────────────────────────────


{
"languages": [
"actions",
"javascript",
"javascript-typescript",
"python",
"rust",
"typescript"
],
"query_suite": "extended",
"runner_label": null,
"runner_type": "standard",
"schedule": null,
"state": "not-configured",
"threat_model": "remote",
"updated_at": null
}
────────────────────────────────────────────────────────────
⚡ Running: gh run list --repo timothywarner-org/globomantics-robot-fleet --workflow=codeql.yml --limit=5
────────────────────────────────────────────────────────────


⚠️ HTTP 404: workflow codeql.yml not found on the default branch (https://api.github.com/repos/timothywarner-org/globomantics-robot-fleet/actions/workflows/codeql.yml)
────────────────────────────────────────────────────────────


CompletedProcess(args='gh run list --repo timothywarner-org/globomantics-robot-fleet --workflow=codeql.yml --limit=5', returncode=1, stdout='', stderr='HTTP 404: workflow codeql.yml not found on the default branch (https://api.github.com/repos/timothywarner-org/globomantics-robot-fleet/actions/workflows/codeql.yml)\n')

## 📋 List All Code Scanning Alerts

The `code-scanning/alerts` endpoint returns every alert found by any scanner configured on the repo. Each alert includes the rule ID, tool name, severity, and current state (open or dismissed).

In [4]:
# List all code scanning alerts with key fields
run_gh(
    f'gh api repos/{REPO}/code-scanning/alerts '
    f'--jq ".[] | {{number, rule: .rule.id, tool: .tool.name, severity: .rule.severity, state: .state}}"'
)

⚡ Running: gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts --jq ".[] | {number, rule: .rule.id, tool: .tool.name, severity: .rule.severity, state: .state}"
────────────────────────────────────────────────────────────


{
"number": 4,
"rule": "rust.lang.security.args.args",
"severity": "note",
"state": "open",
"tool": "Semgrep OSS"
}
{
"number": 3,
"rule": "js/code-injection",
"severity": "error",
"state": "open",
"tool": "CodeQL"
}
{
"number": 2,
"rule": "js/insecure-helmet-configuration",
"severity": "error",
"state": "open",
"tool": "CodeQL"
}
{
"number": 1,
"rule": "js/clear-text-cookie",
"severity": "warning",
"state": "open",
"tool": "CodeQL"
}
────────────────────────────────────────────────────────────


CompletedProcess(args='gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts --jq ".[] | {number, rule: .rule.id, tool: .tool.name, severity: .rule.severity, state: .state}"', returncode=0, stdout='\x1b{\x1b[m\n\x1b"number"\x1b[m\x1b:\x1b[m 4\x1b,\x1b[m\n\x1b"rule"\x1b[m\x1b:\x1b[m \x1b"rust.lang.security.args.args"\x1b[m\x1b,\x1b[m\n\x1b"severity"\x1b[m\x1b:\x1b[m \x1b"note"\x1b[m\x1b,\x1b[m\n\x1b"state"\x1b[m\x1b:\x1b[m \x1b"open"\x1b[m\x1b,\x1b[m\n\x1b"tool"\x1b[m\x1b:\x1b[m \x1b"Semgrep OSS"\x1b[m\n\x1b}\x1b[m\n\x1b{\x1b[m\n\x1b"number"\x1b[m\x1b:\x1b[m 3\x1b,\x1b[m\n\x1b"rule"\x1b[m\x1b:\x1b[m \x1b"js/code-injection"\x1b[m\x1b,\x1b[m\n\x1b"severity"\x1b[m\x1b:\x1b[m \x1b"error"\x1b[m\x1b,\x1b[m\n\x1b"state"\x1b[m\x1b:\x1b[m \x1b"open"\x1b[m\x1b,\x1b[m\n\x1b"tool"\x1b[m\x1b:\x1b[m \x1b"CodeQL"\x1b[m\n\x1b}\x1b[m\n\x1b{\x1b[m\n\x1b"number"\x1b[m\x1b:\x1b[m 2\x1b,\x1b[m\n\x1b"rule"\x1b[m\x1b:\x1b[m \x1b"js/insecure-helmet-configuration"\x1b[m\x1b,\x1b[m\n\x1b"s

## 🔍 Filter Alerts by Severity

Code scanning maps severities as follows:
- **error** = high / critical findings — prioritize these
- **warning** = medium findings
- **note** = low findings

> **💡 GH-500 Exam Tip:** Severity `error` maps to high/critical. `warning` is medium. `note` is low. Triage by severity to focus on what matters most.

In [5]:
# Count of high/critical alerts (severity = error)
run_gh(
    f'gh api repos/{REPO}/code-scanning/alerts '
    f'--jq "[.[] | select(.rule.severity == \\"error\\")] | length"'
)

# List high/critical alerts with rule IDs
run_gh(
    f'gh api repos/{REPO}/code-scanning/alerts '
    f'--jq ".[] | select(.rule.severity == \\"error\\") | {{number, rule: .rule.id}}"'
)

# Warning-level alerts
run_gh(
    f'gh api repos/{REPO}/code-scanning/alerts '
    f'--jq ".[] | select(.rule.severity == \\"warning\\") | {{number, rule: .rule.id}}"'
)

⚡ Running: gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts --jq "[.[] | select(.rule.severity == \"error\")] | length"
────────────────────────────────────────────────────────────


2
────────────────────────────────────────────────────────────
⚡ Running: gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts --jq ".[] | select(.rule.severity == \"error\") | {number, rule: .rule.id}"
────────────────────────────────────────────────────────────


{
"number": 3,
"rule": "js/code-injection"
}
{
"number": 2,
"rule": "js/insecure-helmet-configuration"
}
────────────────────────────────────────────────────────────
⚡ Running: gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts --jq ".[] | select(.rule.severity == \"warning\") | {number, rule: .rule.id}"
────────────────────────────────────────────────────────────


{
"number": 1,
"rule": "js/clear-text-cookie"
}
────────────────────────────────────────────────────────────


CompletedProcess(args='gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts --jq ".[] | select(.rule.severity == \\"warning\\") | {number, rule: .rule.id}"', returncode=0, stdout='\x1b{\x1b[m\n\x1b"number"\x1b[m\x1b:\x1b[m 1\x1b,\x1b[m\n\x1b"rule"\x1b[m\x1b:\x1b[m \x1b"js/clear-text-cookie"\x1b[m\n\x1b}\x1b[m\n', stderr='')

## 🔍 Filter Alerts by Tool

When multiple scanners (CodeQL, Semgrep, etc.) report results to the same repo, you can filter by tool name to see which scanner found each issue. This is essential for multi-scanner repos.

In [6]:
# CodeQL alerts only
print("=== CodeQL Alerts ===")
run_gh(
    f'gh api repos/{REPO}/code-scanning/alerts '
    f'--jq ".[] | select(.tool.name == \\"CodeQL\\") | {{number, rule: .rule.id}}"'
)

# Semgrep alerts only (tool name is "Semgrep OSS" in the API)
print("\n=== Semgrep OSS Alerts ===")
run_gh(
    f'gh api repos/{REPO}/code-scanning/alerts '
    f'--jq ".[] | select(.tool.name == \\"Semgrep OSS\\") | {{number, rule: .rule.id}}"'
)

=== CodeQL Alerts ===
⚡ Running: gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts --jq ".[] | select(.tool.name == \"CodeQL\") | {number, rule: .rule.id}"
────────────────────────────────────────────────────────────


{
"number": 3,
"rule": "js/code-injection"
}
{
"number": 2,
"rule": "js/insecure-helmet-configuration"
}
{
"number": 1,
"rule": "js/clear-text-cookie"
}
────────────────────────────────────────────────────────────

=== Semgrep OSS Alerts ===
⚡ Running: gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts --jq ".[] | select(.tool.name == \"Semgrep OSS\") | {number, rule: .rule.id}"
────────────────────────────────────────────────────────────


{
"number": 4,
"rule": "rust.lang.security.args.args"
}
────────────────────────────────────────────────────────────


CompletedProcess(args='gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts --jq ".[] | select(.tool.name == \\"Semgrep OSS\\") | {number, rule: .rule.id}"', returncode=0, stdout='\x1b{\x1b[m\n\x1b"number"\x1b[m\x1b:\x1b[m 4\x1b,\x1b[m\n\x1b"rule"\x1b[m\x1b:\x1b[m \x1b"rust.lang.security.args.args"\x1b[m\n\x1b}\x1b[m\n', stderr='')

## 📊 Alert Detail — Show Paths & Data Flow

**Path-problem** queries trace data flow from a **source** (where user input enters) through intermediate nodes to a **sink** (where the dangerous operation occurs). An unbroken path from source to sink is a confirmed true positive.

Key vulnerable patterns in this repo:

| Alert | Type | Show Paths? |
|-------|------|-------------|
| `eval()` code injection | Path-problem | Yes — traces `:format` param to `eval()` |
| lodash merge mass assignment | Path-problem | Yes — traces `req.body` to `_.merge()` |
| Hardcoded session secret | Problem (no path) | No |

> **💡 GH-500 Exam Tip:** Show paths traces source to sink. Use it to validate true positives and explain to developers *why* code is vulnerable, not just *where*.

In [7]:
# Get detailed info for alert #1 (adjust number as needed)
run_gh(
    f'gh api repos/{REPO}/code-scanning/alerts/1 '
    f'--jq "{{number, rule: .rule.id, severity: .rule.security_severity_level, '
    f'description: .rule.description, tool: .tool.name, state: .state, html_url: .html_url}}"'
)

# Get alert instances/locations for alert #1
run_gh(
    f'gh api repos/{REPO}/code-scanning/alerts/1/instances '
    f'--jq ".[] | {{ref: .ref, state: .state, location: .location}}"'
)

⚡ Running: gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts/1 --jq "{number, rule: .rule.id, severity: .rule.security_severity_level, description: .rule.description, tool: .tool.name, state: .state, html_url: .html_url}"
────────────────────────────────────────────────────────────


{
"description": "Clear text transmission of sensitive cookie",
"html_url": "https://github.com/timothywarner-org/globomantics-robot-fleet/security/code-scanning/1",
"number": 1,
"rule": "js/clear-text-cookie",
"severity": "medium",
"state": "open",
"tool": "CodeQL"
}
────────────────────────────────────────────────────────────
⚡ Running: gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts/1/instances --jq ".[] | {ref: .ref, state: .state, location: .location}"
────────────────────────────────────────────────────────────


{
"location": {
"end_column": 3,
"end_line": 32,
"path": "server.js",
"start_column": 9,
"start_line": 27
},
"ref": "refs/heads/main",
"state": "open"
}
────────────────────────────────────────────────────────────


CompletedProcess(args='gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts/1/instances --jq ".[] | {ref: .ref, state: .state, location: .location}"', returncode=0, stdout='\x1b{\x1b[m\n\x1b"location"\x1b[m\x1b:\x1b[m \x1b{\x1b[m\n\x1b"end_column"\x1b[m\x1b:\x1b[m 3\x1b,\x1b[m\n\x1b"end_line"\x1b[m\x1b:\x1b[m 32\x1b,\x1b[m\n\x1b"path"\x1b[m\x1b:\x1b[m \x1b"server.js"\x1b[m\x1b,\x1b[m\n\x1b"start_column"\x1b[m\x1b:\x1b[m 9\x1b,\x1b[m\n\x1b"start_line"\x1b[m\x1b:\x1b[m 27\n\x1b}\x1b[m\x1b,\x1b[m\n\x1b"ref"\x1b[m\x1b:\x1b[m \x1b"refs/heads/main"\x1b[m\x1b,\x1b[m\n\x1b"state"\x1b[m\x1b:\x1b[m \x1b"open"\x1b[m\n\x1b}\x1b[m\n', stderr='')

## ⚡ Alert Triage — Dismiss & Reopen

Not every alert demands immediate remediation. The dismissal workflow lets you document decisions and creates an **audit trail**.

Three dismissal reasons:
1. **False positive** — the scanner got it wrong
2. **Won't fix** — real issue but accepted risk
3. **Used in tests** — intentionally vulnerable test/demo code

> **💡 GH-500 Exam Tip:** Dismissals require documentation. Know all three reasons: false positive, won't fix, used in tests. All create audit trails. Your dismissal comment should be defensible — "I didn't feel like fixing it" won't fly with auditors.

In [8]:
# ======================================================================
# DISMISS AN ALERT
# ======================================================================
# UNCOMMENT the lines below to dismiss — replace alert number as needed
#
# run_gh(
#     f'gh api repos/{REPO}/code-scanning/alerts/1 '
#     f'--method PATCH '
#     f'--field state=dismissed '
#     f'--field dismissed_reason="used in tests" '
#     f'--field dismissed_comment="Educational demo repo with intentional vulnerabilities"'
# )
#
# # Verify the dismissal
# run_gh(
#     f'gh api repos/{REPO}/code-scanning/alerts/1 '
#     f'--jq "{{number, state, dismissed_reason, dismissed_comment}}"'
# )

# ======================================================================
# RE-OPEN A DISMISSED ALERT
# ======================================================================
# UNCOMMENT the lines below to re-open — replace alert number as needed
#
# run_gh(
#     f'gh api repos/{REPO}/code-scanning/alerts/1 '
#     f'--method PATCH '
#     f'--field state=open'
# )
#
# # Verify re-open
# run_gh(
#     f'gh api repos/{REPO}/code-scanning/alerts/1 '
#     f'--jq "{{number, state, dismissed_reason}}"'
# )

print("\u2139\ufe0f  Dismiss/reopen commands are commented out to prevent accidental execution.")
print("   Uncomment the section you need and update the alert number.")

ℹ️  Dismiss/reopen commands are commented out to prevent accidental execution.
   Uncomment the section you need and update the alert number.


## 🚀 Workflow Status & Logs

When troubleshooting, start with the workflow status. Is it passing, failing, or stuck? The `gh` CLI gives you a quick summary. The logs tell you everything — permission failures show 403, build failures show compiler errors, extraction issues list which files could not be processed.

In [9]:
# List recent workflow runs (all workflows)
run_gh(f'gh run list --repo {REPO} --limit=5')

# List CodeQL-specific runs
run_gh(f'gh run list --repo {REPO} --workflow=codeql.yml --limit=5')

# List Semgrep-specific runs
run_gh(f'gh run list --repo {REPO} --workflow=semgrep-analysis.yml --limit=5')

⚡ Running: gh run list --repo timothywarner-org/globomantics-robot-fleet --limit=5
────────────────────────────────────────────────────────────


completed	failure	npm_and_yarn in /. for axios, axios, axios, axios, body-parser, braces, debug, debug, dicer, elliptic, express, express, form-data, got, handlebars, handlebars, handlebars, handlebars, handlebars, handlebars, handlebars, handlebars, handlebars, handlebars, jsonwebtoken, jsonwebtoken, jsonwebtoken, lodash, lodash, lodash, minimatch, minimist, minimist, moment, moment, multer, path-to-regexp, path-to-regexp, qs, qs, qs, request, semver, send, serialize-javascript, serve-static, sqlite3, sqlite3, tar, tar, ...	Dependabot Updates	main	dynamic	22234835209	2m0s	2026-02-20T17:48:18Z
completed	success	feat: add Semgrep workflow, demo scripts, and Jupyter notebooks	Semgrep Security Analysis	main	push	22234831224	33s	2026-02-20T17:48:10Z
completed	success	npm_and_yarn in / for cookie-parser, express-session, lodash, moment, sqlite3, validator, cors, handlebars, are-we-there-yet, async, core-util-is, detect-libc, gauge, minimist, node-gyp, nopt, npmlog, on-headers, uglify-js, wo

⚠️ HTTP 404: workflow codeql.yml not found on the default branch (https://api.github.com/repos/timothywarner-org/globomantics-robot-fleet/actions/workflows/codeql.yml)
────────────────────────────────────────────────────────────
⚡ Running: gh run list --repo timothywarner-org/globomantics-robot-fleet --workflow=semgrep-analysis.yml --limit=5
────────────────────────────────────────────────────────────


completed	success	feat: add Semgrep workflow, demo scripts, and Jupyter notebooks	Semgrep Security Analysis	main	push	22234831224	33s	2026-02-20T17:48:10Z
────────────────────────────────────────────────────────────


CompletedProcess(args='gh run list --repo timothywarner-org/globomantics-robot-fleet --workflow=semgrep-analysis.yml --limit=5', returncode=0, stdout='completed\tsuccess\tfeat: add Semgrep workflow, demo scripts, and Jupyter notebooks\tSemgrep Security Analysis\tmain\tpush\t22234831224\t33s\t2026-02-20T17:48:10Z\n', stderr='')

## 🎯 Upload SARIF via CodeQL CLI

External CI systems (Jenkins, Azure DevOps, GitLab CI) can upload SARIF results to GitHub using the CodeQL CLI or the `upload-sarif` action. The `--sarif-category` flag is **required** when multiple scanners report results — without it, uploads overwrite each other.

> **💡 GH-500 Exam Tip:** Third-party scanners use `upload-sarif`, NOT `codeql-action/analyze`. SARIF 2.1.0 is the only supported version. Categories prevent result overwrites — always set `--sarif-category` or the `category` input when using multiple scanners.

In [10]:
# ======================================================================
# UPLOAD SARIF FILE VIA CODEQL CLI
# ======================================================================
# UNCOMMENT to upload — requires a .sarif file to exist in the repo root
#
# commit_sha = subprocess.run(
#     ["git", "rev-parse", "HEAD"],
#     capture_output=True, text=True, cwd=REPO_DIR
# ).stdout.strip()
#
# run_gh(
#     f'codeql github upload-results '
#     f'--repository={REPO} '
#     f'--ref=refs/heads/main '
#     f'--commit={commit_sha} '
#     f'--sarif=rust-scan.sarif '
#     f'--sarif-category=semgrep-rust-local',
#     cwd=REPO_DIR
# )

print("\u2139\ufe0f  Upload command is commented out. Uncomment after generating a .sarif file.")
print("   Example: semgrep scan --config p/rust --sarif --output rust-scan.sarif ./rust-telemetry-cli")

ℹ️  Upload command is commented out. Uncomment after generating a .sarif file.
   Example: semgrep scan --config p/rust --sarif --output rust-scan.sarif ./rust-telemetry-cli


## 💡 GH-500 Exam Quick Reference

| Topic | Key Point |
|-------|----------|
| Query suites | default = fewer queries, security-extended = comprehensive |
| Default setup vs default suite | Different things — default setup is zero-config enablement |
| Build modes | `none` (interpreted), `autobuild` (auto-detect), `manual` (explicit) |
| Show paths | Traces source to sink — validates true positives |
| Dismissal reasons | false positive, won't fix, used in tests — all create audit trails |
| SARIF upload | Uses `upload-sarif` action (part of `codeql-action` repo) |
| Categories | REQUIRED for multiple scanners — prevents overwrites |
| Copilot Autofix | Ships with GHAS, no separate Copilot subscription required |
| Copilot Chat | Requires Copilot Enterprise license (separate from GHAS) |
| Troubleshooting | 90% of failures = not explicit enough in configuration |

### Common Failure Scenarios

| Error | Cause | Fix |
|-------|-------|-----|
| Language detection failed | Auto-detection missed a language | Specify `languages:` explicitly in workflow |
| Autobuild failed | Non-standard build system | Use `build-mode: manual` with explicit commands |
| Timeout exceeded | Large codebase | Increase `timeout-minutes`, split into matrix jobs |
| Permission denied (403) | Missing workflow permission | Add `security-events: write` |
| No results returned | Extraction failed silently | Check logs, switch to `security-extended` suite |